<a href="https://colab.research.google.com/github/Nobu90/scaling-broccoli/blob/main/%5BE_06%5Dlyricist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 루브릭

## 1. 데이터의 전처리 및 구성과정이 체계적으로 진행되었는가?

특수문자 제거, 토크나이저 생성, 패딩 처리의 작업들이 빠짐없이 진행되었는가?
## 2. 가사 텍스트 생성 모델이 정상적으로 동작하는가?

텍스트 제너레이션 결과로 생성된 문장이 해석 가능한 문장인가?
## 3. 텍스트 생성모델이 안정적으로 학습되었는가?

텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Libraries

In [2]:
import glob
import os, re
import tensorflow as tf
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential

print(tf.__version__)

2.9.2


# Load data

In [3]:
txt_file_path = '/content/drive/MyDrive/Exploration/RS2/data/lyrics/*' 

txt_list = glob.glob(txt_file_path) 

raw_corpus = [] 

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() 
        raw_corpus.extend(raw) 

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 189090
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


# Data Preprocessing

In [4]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", sentence)
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence <end>


In [5]:
corpus = []

for sentence in raw_corpus:

    if len(sentence) == 0: continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:20]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby cant you read the signs i wont bore you with the details baby <end>',
 '<start> i dont even wanna waste your time <end>',
 '<start> lets just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i aint mr right but if youre looking for fast love <end>',
 '<start> if thats love in your eyes <end>',
 '<start> its more than enough <end>',
 '<start> had some bad love <end>',
 '<start> so fast love is all that ive got on my mind ooh ooh <end>',
 '<start> ooh ooh looking for some affirmation <end>',
 '<start> made my way into the sun <end>',
 '<start> my friends got their ladies <end>',
 '<start> and theyre all having babies <end>',
 '<start> i just wanna have some fun i wont bore you with the details baby <end>',
 '<start> i dont even wanna waste your time <end>',
 '<start> lets just say that maybe <end>',
 '<s

# Tokenization

In [6]:
def tokenize(corpus):
 
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 306  23 ...   0   0   0]
 [  2 215  11 ...   0   0   0]
 [  2  20  15 ...   0   0   0]
 ...
 [  2  22  71 ...   0   0   0]
 [  2  35  21 ...   0   0   0]
 [  2  22  71 ...   0   0   0]] <keras.preprocessing.text.Tokenizer object at 0x7fbb23ca84c0>


In [7]:
print(tensor[:3, :15])

[[   2  306   23   90 4877    3    0    0    0    0    0    0    0    0
     0]
 [   2  215   11   79  224    4  114    3    0    0    0    0    0    0
     0]
 [   2   20   15 1037 2491    3    0    0    0    0    0    0    0    0
     0]]


In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : the
5 : i
6 : you
7 : and
8 : to
9 : a
10 : me


In [9]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])
print(src_input.shape)
print(tgt_input.shape)

[   2  306   23   90 4877    3    0    0    0    0    0    0    0    0]
[ 306   23   90 4877    3    0    0    0    0    0    0    0    0    0]
(177988, 14)
(177988, 14)


# Train data

In [10]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=999, shuffle='true')
print(enc_train.shape)
print(dec_train.shape)
print(enc_val.shape)
print(dec_val.shape)

(142390, 14)
(142390, 14)
(35598, 14)
(35598, 14)


In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 512
hidden_size = 2048

lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

## 1. 최근에는 임베딩 사이즈를 512, 768, 1024 등으로 과거보다 크게 주는 경향이 크다고 함

### [출처]
https://ai.stackexchange.com/questions/28564/how-to-determine-the-embedding-size

## 2. 배치 사이즈와, 임베딩 사이즈로는 로스 값이 줄지 않아 히든 사이즈를 2배로 키운 결과 결과가 좋아짐, 레이어의 수를 늘려보았으나 크게 영향을 미치지는 않았음

In [12]:
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, 
    reduction='none'  
)

lyricist.compile(loss=loss, optimizer=optimizer) 

In [13]:
history = lyricist.fit(enc_train, dec_train, validation_data = (enc_val, dec_val), epochs = 10, batch_size = 256, validation_batch_size = 128)


Epoch 1/10
557/557 [==============================] - 48s 41ms/step - loss: 3.4648 - val_loss: 3.1180
Epoch 2/10
557/557 [==============================] - 22s 39ms/step - loss: 2.9394 - val_loss: 2.8494
Epoch 3/10
557/557 [==============================] - 22s 39ms/step - loss: 2.6550 - val_loss: 2.6647
Epoch 4/10
557/557 [==============================] - 22s 39ms/step - loss: 2.3834 - val_loss: 2.5203
Epoch 5/10
557/557 [==============================] - 22s 39ms/step - loss: 2.1216 - val_loss: 2.4078
Epoch 6/10
557/557 [==============================] - 22s 39ms/step - loss: 1.8730 - val_loss: 2.3178
Epoch 7/10
557/557 [==============================] - 22s 39ms/step - loss: 1.6429 - val_loss: 2.2547
Epoch 8/10
557/557 [==============================] - 22s 39ms/step - loss: 1.4400 - val_loss: 2.2102
Epoch 9/10
557/557 [==============================] - 22s 39ms/step - loss: 1.2687 - val_loss: 2.1870
Epoch 10/10
557/557 [==============================] - 22s 39ms/step - loss: 1.133

In [14]:
results = lyricist.evaluate(enc_val, dec_val)

1113/1113 [==============================] - 8s 7ms/step - loss: 2.1863


## 1. 적절한 배치 사이즈는 일반적으로 16 ~ 128로 2의 제곱으로 넣으나 가장 최적의 배치 사이즈란 정해져 있지 않다고 함 
## (테스트 결과 여기서는 각 256, 128이 적절하였음, 키울 수록 과적합, 줄일 수록 로스가 커지는 현상이 나타남)

### [출처]
https://stats.stackexchange.com/questions/308424/how-does-batch-size-affect-adam-optimizer

https://deep-learning-study.tistory.com/647

# Generate Lyrics

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): 

    test_input = tokenizer.texts_to_sequences([init_sentence]) 
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True: 
    
        predict = model(test_tensor) 
  
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated 

In [18]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)


'<start> i love you so much i love you so much <end> '

# 후기

코랩 프로 플러스의 속도를 체감할 수 있었다. 앞으로도 계속 유료 결제를 떠날 수 없을 것 같다.
이번 익스는 조건이 좀 명확하게 있어서 정확하게 조건을 맞추느라 까다로웠으나, 실제 공부에는 도움이 많이 되었던 것 같다.
덕분에 배치사이즈에 대해서는 정말 많이 검색해본 것 같고, 이번 익스에서 느낀 점은 역시나 단 한 번의 터치, 옵션 변경으로 결과가 드라마틱하게 좋아진다는 건 존재하지 않는 다는 걸 깨달았다. 
데이터, 그리고 내가 사용하는 모델과 그 안의 옵션, 파라미터들을 더 정확하고 자세히 알고, 각 원리도 총체적으로 이해해야 원하는 결과를 빠르게 얻을 수 있다는 생각을 다시 하게 되었다.
그리고 빨리 얻지 못하면, 구글에 계속 돈을 내야 한다는 사실도... 
그래도 결과도 결과지만 마지막에는 처음에 나왔던 가사들보다 자연스러운 가사가 나와서 기뻤다(후렴구인듯..)
